In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
# ฟังก์ชันโหลดเสียงและแปลงเป็น Mel Spectrogram
def extract_features(audio_file):
    try:
        y, sr = librosa.load(audio_file, sr=None)
        if y is None or len(y) == 0:
            raise ValueError(f"ไฟล์ {audio_file} ไม่มีข้อมูลเสียง")

        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
        log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        return log_mel_spec
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการโหลดไฟล์ {audio_file}: {e}")
        return None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
dog_dir = "/content/drive/MyDrive/dog_wav"
cat_dir = "/content/drive/MyDrive/cat_wav"

# ตรวจสอบว่าไดเรกทอรีมีอยู่
if os.path.exists(dog_dir):
    print(f"Dog directory exists: {dog_dir}")
else:
    print(f"Dog directory does not exist: {dog_dir}")

if os.path.exists(cat_dir):
    print(f"Cat directory exists: {cat_dir}")
else:
    print(f"Cat directory does not exist: {cat_dir}")

# ฟังก์ชันดึงคุณลักษณะจากไฟล์เสียง
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)  # โหลดไฟล์เสียง
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)  # แปลงเป็น Mel Spectrogram
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # ทำให้เป็น dB scale
        return mel_spec_db
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# อ่านไฟล์ทั้งหมดจากไดเรกทอรี
dog_files = [os.path.join(dog_dir, f) for f in os.listdir(dog_dir) if f.endswith('.wav')]
cat_files = [os.path.join(cat_dir, f) for f in os.listdir(cat_dir) if f.endswith('.wav')]

# รวมไฟล์เสียงและแท็ก
features, labels = [], []

# สำหรับไฟล์เสียงหมา
for file in dog_files:
    mel_spec = extract_features(file)
    if mel_spec is not None:
        features.append(mel_spec)
        labels.append(0)  # 0 = dog

# สำหรับไฟล์เสียงแมว
for file in cat_files:
    mel_spec = extract_features(file)
    if mel_spec is not None:
        features.append(mel_spec)
        labels.append(1)  # 1 = cat

if not features:
    raise ValueError("ไม่สามารถโหลดไฟล์เสียงได้ ตรวจสอบไฟล์ต้นฉบับอีกครั้ง")

Dog directory exists: /content/drive/MyDrive/dog_wav
Cat directory exists: /content/drive/MyDrive/cat_wav


In [ ]:
max_length = max([feature.shape[1] for feature in features])  # หาขนาดสูงสุด
features_padded = [np.pad(feature, ((0, 0), (0, max_length - feature.shape[1])), mode='constant') for feature in features]

X = np.array(features_padded)
y = np.array(labels)

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# เปลี่ยนรูปร่างของ X_train และ X_test ให้มีมิติใหม่
X_train, X_test = X_train[..., np.newaxis], X_test[..., np.newaxis]


In [ ]:
# สร้างโมเดล CNN
model = models.Sequential([
    layers.InputLayer(shape=(X_train.shape[1], X_train.shape[2], 1)),  # ใช้ 'shape' แทน 'input_shape'

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # ใช้ sigmoid เนื่องจากเป็นปัญหาการจำแนก 2 คลาส
])

# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 484ms/step - accuracy: 0.5193 - loss: 8.3797 - val_accuracy: 0.6667 - val_loss: 0.6830
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.7547 - loss: 0.4917 - val_accuracy: 0.7708 - val_loss: 0.4609
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8974 - loss: 0.2888 - val_accuracy: 0.7917 - val_loss: 0.5051
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9042 - loss: 0.1962 - val_accuracy: 0.7917 - val_loss: 0.7131
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9554 - loss: 0.1095 - val_accuracy: 0.7917 - val_loss: 0.8929
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9752 - loss: 0.0598 - val_accuracy: 0.8542 - val_loss: 0.7929
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9728 - loss: 0.0722 - val_accuracy: 0.8333 - val_loss: 0.9953
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9914 - loss: 0.0230 - val_accuracy: 

In [ ]:
predictions = model.predict(X_test)

for i in range(min(5, len(X_test))):
    cat_prob = predictions[i][0] * 100  # เปลี่ยนเป็นเปอร์เซ็นต์
    dog_prob = 100 - dog_prob  # คำนวณเปอร์เซ็นต์ของ Music
    print(f"ตัวอย่าง {i+1}: dog = {dog_prob:.2f}%, cat = {cat_prob:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
ตัวอย่าง 1: dog = 100.00%, cat = 100.00%
ตัวอย่าง 2: dog = 0.00%, cat = 0.00%
ตัวอย่าง 3: dog = 100.00%, cat = 0.25%
ตัวอย่าง 4: dog = 0.00%, cat = 100.00%
ตัวอย่าง 5: dog = 100.00%, cat = 0.01%


In [ ]:
model.save("dog.h5")
print("โมเดลถูกบันทึกเรียบร้อยแล้ว 🎉")


โมเดลถูกบันทึกเรียบร้อยแล้ว 🎉


In [ ]:
import os
import librosa
import numpy as np
from tensorflow.keras.models import load_model

# โหลดโมเดลที่ฝึกไว้
model = load_model("dog.h5")
print("✅ โหลดโมเดลสำเร็จ")

# ฟังก์ชันแปลงไฟล์เสียงเป็น Mel Spectrogram
def extract_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    return log_mel_spec

# ระบุพาธไฟล์เสียงที่ต้องการทดสอบ
test_audio = "cat_36.wav"

if not os.path.exists(test_audio):
    raise ValueError(f"❌ ไม่พบไฟล์ {test_audio} โปรดตรวจสอบพาธอีกครั้ง")

# ดึง features จากไฟล์เสียง
mel_spec = extract_features(test_audio)
#print(model.input_shape)

# แสดงขนาดของ Mel Spectrogram ก่อน Padding
print(f"ขนาดของ Mel Spectrogram ก่อน Padding: {mel_spec.shape}")

# ปรับขนาด max_len ให้เหมาะสมกับข้อมูลที่ใช้ฝึกโมเดล
max_len = 553  # ขนาดความยาวที่โมเดลคาดหวัง

# ทำการ Padding หรือครอบขนาดให้ตรงกับ max_len
if mel_spec.shape[1] < max_len:
    mel_spec = np.pad(mel_spec, ((0, 0), (0, max_len - mel_spec.shape[1])))
elif mel_spec.shape[1] > max_len:
    mel_spec = mel_spec[:, :max_len]

# แสดงขนาดของ Mel Spectrogram หลังจาก Padding
print(f"ขนาดของ Mel Spectrogram หลัง Padding: {mel_spec.shape}")

# ปรับขนาดให้เหมาะสมกับโมเดล (เพิ่มมิติช่องสัญญาณ)
mel_spec = np.expand_dims(mel_spec, axis=-1)  # (128, 1320, 1)

# ตรวจสอบข้อมูล
print(f"ข้อมูลหลังปรับขนาด: {mel_spec.shape}")

# ทำการทำนาย
prediction = model.predict(np.expand_dims(mel_spec, axis=0))[0][0]

# แสดงผลลัพธ์เป็นเปอร์เซ็นต์
cat_prob = prediction * 100
dog_prob = (1 - prediction) * 100
print(f"🔊 ผลการทำนาย: dog {dog_prob:.2f}% | cat {cat_prob:.2f}%")


✅ โหลดโมเดลสำเร็จ
ขนาดของ Mel Spectrogram ก่อน Padding: (128, 376)
ขนาดของ Mel Spectrogram หลัง Padding: (128, 553)
ข้อมูลหลังปรับขนาด: (128, 553, 1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
🔊 ผลการทำนาย: dog 0.00% | cat 100.00%
